In [2]:
import pandas as pd
import json

# 1. 데이터 로드
with open('kmooc_courses.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

In [3]:

# 2. 데이터 기본 정보 확인
print("--- [1] 데이터 기본 구조 ---")
df.info()  # 전체 행/열 수, 데이터 타입 확인

--- [1] 데이터 기본 구조 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15522 entries, 0 to 15521
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   15522 non-null  int64  
 1   shortname            15522 non-null  object 
 2   name                 15522 non-null  object 
 3   url                  15522 non-null  object 
 4   course_image         15522 non-null  object 
 5   org                  15522 non-null  object 
 6   org_name             15522 non-null  object 
 7   enrollment_start     15521 non-null  object 
 8   enrollment_end       15521 non-null  object 
 9   study_start          15522 non-null  object 
 10  study_end            15522 non-null  object 
 11  professor            14061 non-null  object 
 12  public_yn            15522 non-null  object 
 13  summary              6994 non-null   object 
 14  classfy_name         7038 non-null   object 
 15  middle_classfy

In [4]:

# 3. 결측치(Null) 확인
print("\n--- [2] 결측치 현황 ---")
print(df.isnull().sum())



--- [2] 결측치 현황 ---
id                         0
shortname                  0
name                       0
url                        0
course_image               0
org                        0
org_name                   0
enrollment_start           1
enrollment_end             1
study_start                0
study_end                  0
professor               1461
public_yn                  0
summary                 8528
classfy_name            8484
middle_classfy_name     8484
week                    8484
course_playtime         8484
detail_error_raw       15522
dtype: int64


In [5]:
# 4. 수치형 데이터 통계 (주차, 총 학습시간)
print("\n--- [3] 주요 수치 통계 ---")
# 문자열로 저장되어 있을 수 있으므로 숫자형으로 변환 후 요약
df['week'] = pd.to_numeric(df['week'], errors='coerce')
df['course_playtime'] = pd.to_numeric(df['course_playtime'], errors='coerce')
print(df[['week', 'course_playtime']].describe())



--- [3] 주요 수치 통계 ---
              week  course_playtime
count  7038.000000      7038.000000
mean     11.214692     69029.667519
std       4.940266     49078.390656
min       1.000000         0.000000
25%       8.000000     36000.000000
50%      14.000000     57600.000000
75%      15.000000     97200.000000
max      22.000000    457080.000000


In [6]:
# 5. 카테고리별 데이터 분포
print("\n--- [4] 대분류별 강좌 수 ---")
print(df['classfy_name'].value_counts())

print("\n--- [5] 상위 10개 제공 기관 ---")
print(df['org_name'].value_counts().head(10))



--- [4] 대분류별 강좌 수 ---
classfy_name
인문        1833
공학        1628
사회        1625
자연         489
예체능        481
의약         460
교육         295
기타         114
융 · 복합     113
Name: count, dtype: int64

--- [5] 상위 10개 제공 기관 ---
org_name
EBS        638
서울대학교      582
성균관대학교     504
고려대학교      479
이화여자대학교    406
성신여자대학교    345
단국대학교      337
대구대학교      330
부산대학교      299
경희대학교      242
Name: count, dtype: int64


In [7]:
# 6. 공개 여부 확인
print("\n--- [6] 공개 여부(public_yn) 비율 ---")
print(df['public_yn'].value_counts(normalize=True) * 100)


--- [6] 공개 여부(public_yn) 비율 ---
public_yn
N    54.64502
Y    45.35498
Name: proportion, dtype: float64


In [9]:
# 7. 텍스트 임베딩 및 추천 시스템용 데이터 진단
print("--- [1] 텍스트 임베딩 핵심 필드 진단 ---")
# 텍스트가 있어야 임베딩이 가능하므로, 필수 필드 결합
df['combined_text'] = df['name'].fillna('') + " " + df['summary'].fillna('')

# 7-1. 텍스트 길이가 너무 짧은 경우 (임베딩 의미 없음)
# 이름만 있고 설명이 없는 데이터가 얼마나 되는지 확인
short_text_count = df[df['combined_text'].str.len() < 20].shape[0]
print(f"⚠️ 텍스트 20자 미만 (정보 부족): {short_text_count}건")

--- [1] 텍스트 임베딩 핵심 필드 진단 ---
⚠️ 텍스트 20자 미만 (정보 부족): 6297건


In [10]:
# 7-2. 중복된 강좌명/설명 확인 (추천 결과가 중복될 위험)
duplicate_names = df[df.duplicated(['name'], keep=False)].shape[0]
print(f"⚠️ 중복된 강좌명: {duplicate_names}건 (서로 다른 ID인데 이름이 같은 경우)")

⚠️ 중복된 강좌명: 14124건 (서로 다른 ID인데 이름이 같은 경우)


In [11]:
# 8. 결측치 패턴 분석 (추천 필터링용)
print("\n--- [2] 결측치 패턴 분석 (추천 필터링용) ---")
# 추천 시스템에서 필터링 조건으로 많이 쓰는 필드들
filter_cols = ['classfy_name', 'middle_classfy_name', 'week', 'professor']
missing_patterns = df[filter_cols].isnull().sum()
print(missing_patterns)


--- [2] 결측치 패턴 분석 (추천 필터링용) ---
classfy_name           8484
middle_classfy_name    8484
week                   8484
professor              1461
dtype: int64


In [12]:
# 8-1. '분류'가 아예 없는 데이터 (카테고리 추천 불가)
no_class = df[df['classfy_name'].isnull() & df['middle_classfy_name'].isnull()].shape[0]
print(f"⚠️ 대/중분류 모두 없는 강좌: {no_class}건")

⚠️ 대/중분류 모두 없는 강좌: 8484건


In [13]:
# 9. 데이터 타입 및 특수 기호 점검
print("\n--- [3] 데이터 타입 및 특수 기호 점검 ---")
# 9-1. 수치형 데이터에 0이나 음수가 있는지 (course_playtime 등)
invalid_time = df[df['course_playtime'] <= 0].shape[0]
print(f"⚠️ 학습 시간 0 이하: {invalid_time}건")


--- [3] 데이터 타입 및 특수 기호 점검 ---
⚠️ 학습 시간 0 이하: 67건


In [14]:
# 9-2. 특수문자나 HTML 잔재가 남아있는지 확인 (임베딩 노이즈)
# summary에 '<' 나 '>' 가 포함된 행 추출
html_remnants = df[df['summary'].str.contains('<|>', na=False)].shape[0]
print(f"⚠️ HTML 태그 잔재 의심: {html_remnants}건")

⚠️ HTML 태그 잔재 의심: 775건
